<img src="https://github.com/thesps/conifer/blob/master/conifer_v1.png?raw=true" width="250" alt="conifer" />

In this notebook we will take the first steps with training a BDT with `xgboost`, then translating it to HLS code for FPGA with `conifer`

Key concepts:
- dataset creation
- model training
- model evaluation
- `conifer` configuration and conversion
- model emulation / re-evaluation
- model synthesis

In [ ]:
# INFN CNAF Setup
import os
os.environ['PATH'] = '/tools/Xilinx/Vitis_HLS/2023.2/tps/lnx64/gcc-9.3.0/bin/:' + '/tools/Xilinx/Vitis_HLS/2023.2/bin/:' + os.environ['PATH']
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2/'

In [ ]:
from sklearn.datasets import make_moons
from sklearn.inspection import DecisionBoundaryDisplay
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
import conifer
import json
import os
import sys

# enable more output from conifer
import logging
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logger = logging.getLogger('conifer')
logger.setLevel('INFO')

# create a random seed at we use to make the results repeatable
seed = int('fpga_tutorial'.encode('utf-8').hex(), 16) % 2**31

# Create a dataset
For this first introduction we will create a synthetic random separable dataset with 2 variables from `scikit-learn` `make_moons`. You'll see where it gets the name when we plot it below. We make a crude train/test split, using some of the data for the model training and reserving the rest for testing.

In [ ]:
X, y = make_moons(n_samples=5000, noise=0.3, random_state=seed)
X_train, X_test = X[:2500], X[2500:]
y_train, y_test = y[:2500], y[2500:]

# save to files
os.makedirs('moons_dataset', exist_ok=True)
np.save('moons_dataset/X_train.npy', X_train)
np.save('moons_dataset/X_test.npy', X_test)
np.save('moons_dataset/y_train.npy', y_train)
np.save('moons_dataset/y_test.npy', y_test)

## Plot dataset
Visualise the dataset we just created. There are two variables that we plot on `(x,y)` and two classes that we show with colour.

In [ ]:
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap='PiYG', edgecolors='k')

# Train a BDT
We'll use `xgboost`'s `XGBClassifier` with:

| Parameter | Explanation |
| --- | --- |
| `n_estimators=20` | 20 trees
| `max_depth=3` | maximum tree depth of 3
| `learning_rate=1.0` | |

In [ ]:
clf = xgb.XGBClassifier(n_estimators=20, max_depth=3, learning_rate=1.0,
                        random_state=seed).fit(X_train, y_train)

# Validate performance
Now we check whether the trained model is any good. Firstly we'll visualise the decision boundary which shows how the strength of the prediction varies across the parameter space. Where there is more overlap between the classes, the prediction is less certain.

We plot a few of the test set examples on top.

In [ ]:
DecisionBoundaryDisplay.from_estimator(clf, X_test, cmap='PiYG')
plt.scatter(X_test[:,0][:200], X_test[:,1][:200], c=y_test[:200], cmap='PiYG', edgecolors='k')

<img src="https://github.com/thesps/conifer/blob/master/conifer_v1.png?raw=true" width="250" alt="conifer" />

Now we'll convert this model to FPGA firmware with `conifer`. We first need to create a configuration in the form of a dictionary. The quickest way to get started is to create a default configuration from the intended target backend (`xilinxhls` for us). Each backend may have different configuration options, so getting the configuration this way helps enumerate the possible options.

We will print the configuration, modify it, and print it again. The modifications are:
- set the `OutputDirectory` to something descriptive
- set the `XilinxPart` to the part number of the FPGA on the Alveo U50

In [ ]:
cfg = conifer.backends.xilinxhls.auto_config()

# print the config
print('Default Configuration\n' + '-' * 50)
print(json.dumps(cfg, indent=2))
print('-' * 50)

# modify the config
cfg['OutputDir'] = 'prj_conifer_part_1'
cfg['XilinxPart'] = 'xcu50-fsvh2104-2-e'

# print the config again
print('Modified Configuration\n' + '-' * 50)
print(json.dumps(cfg, indent=2))
print('-' * 50)

## Convert and write
Convert the `xgboost` model to a `conifer` one, and print the `help` to see what methods it implements.
Then `write` the model, creating the specified output directory and writing all the HLS files to it. We also save the `xgboost` model itself.

#### Other converters:
`conifer` has converters for several popular BDT training libraries. Each one is used like: `conifer.converters.convert_from_<library>(model, config)`
The converters are:
- `sklearn`
- `xgboost`
- `tf_df`
- `tmva`
- `onnx`

In [ ]:
conifer_model = conifer.converters.convert_from_xgboost(clf, cfg)
help(conifer_model)
conifer_model.write()
clf.save_model('prj_conifer_part_1/xgboost_model.json')

## Explore
Browse the files in the newly created project directory to take a look at the HLS code.

The output of `!tree prj_conifer_part_1` is:

```
prj_conifer_part_1/
├── bridge.cpp
├── build_hls.tcl
├── firmware
│   ├── BDT.cpp
│   ├── BDT.h
│   ├── my_prj.cpp
│   ├── my_prj.h
│   └── parameters.h
├── hls_parameters.tcl
├── my_prj.json
├── my_prj_test.cpp
├── tb_data
└── vivado_synth.tcl

2 directories, 11 files
```

- files under `firmware` are the HLS implementation of the model
- `my_prj.json` is the saved converted `conifer` model that can be loaded again without the original `xgboost` model
- `tcl` scripts are used for synthesizing the project

## Emulate
Before starting the lengthy FPGA build process, we should validate that our conversion was successful and that the choice of precision was suitable. To do this we need to run the HLS C++ code on the CPU with some test data first. This is like the HLS C Simulation step, but rather than writing a C++ testbench and invoking `vitis_hls` to run `csim`, `conifer` implements Python bindings for the HLS.

We first need to compile (which uses the C++ compiler), then we can make predictions

In [ ]:
conifer_model.compile()

In [ ]:
y_hls = conifer_model.decision_function(X_test)

## Compare
Now we check whether the emulated predictions are good. To do this, first we'll make the decision boundary for both the `conifer` and `xgboost` models, and also show the difference.

In [ ]:
# make a 1000x1000 grid of points in the feature space
X_mesh = np.meshgrid(np.linspace(-3, 3, 1000), np.linspace(-3, 3, 1000))
# reshape them for inference
X_grid = np.vstack([X_mesh[0].ravel(), X_mesh[1].ravel()]).T
# run emulated inference, compute the class probability, reshape to 1000x1000 grid
y_hls_mesh = np.reshape(expit(conifer_model.decision_function(X_grid)), X_mesh[0].shape)
# run the xgboost prediction on the same grid
y_xgb_mesh = np.reshape(clf.predict_proba(X_grid)[:,1], X_mesh[0].shape)

In [ ]:
# display the boundaries, and the difference
f, axs = plt.subplots(1, 3, figsize=(15,5))

# plot HLS
display = DecisionBoundaryDisplay(xx0=X_mesh[0], xx1=X_mesh[1], response=y_hls_mesh)
display.plot(cmap='PiYG', ax=axs[0])
axs[0].scatter(X_test[:,0][:200], X_test[:,1][:200], c=y_test[:200], cmap='PiYG', edgecolors='k')
axs[0].set_title('HLS')

# plot the XGBoost
display = DecisionBoundaryDisplay(xx0=X_mesh[0], xx1=X_mesh[1], response=y_xgb_mesh)
display.plot(cmap='PiYG', ax=axs[1])
axs[1].scatter(X_test[:,0][:200], X_test[:,1][:200], c=y_test[:200], cmap='PiYG', edgecolors='k')
axs[1].set_title('XGBoost')

# plot the difference
pcm = axs[2].pcolormesh(X_mesh[0], X_mesh[1], y_xgb_mesh-y_hls_mesh)
axs[2].set_title('XGBoost - HLS')
f.colorbar(pcm)
plt.tight_layout()

## Build
Now we'll run the Vitis HLS and Vivado synthesis. HLS C Synthesis compiles our C++ to RTL, performing scheduling and resource mapping. Vivado synthesis synthesizes the RTL from the previous step into a netlist, and produces a more realistic resource estimation. The latency can't change during Vivado synthesis, it's fixed in the RTL description.

After the build completes we can also browse the new log files and reports that are generated.

In [ ]:
conifer_model.build(synth=True, vsynth=True)

## Report
If the synthesis completed successfuly, we can extract the key metrics from the reports and print them out.
The section `"vsynth"` contains the report from the Vivado RTL synthesis, which is usually lower, and more realistic than the HLS report.

In [ ]:
report = conifer_model.read_report()
print(json.dumps(report, indent=2))

## Exercise: Improve
The `conifer` model seems to follow the `xgboost` predictions quite well, but it could be better. Our `xgboost` predictions were made with `float` data, thresholds, and scores while the `conifer` HLS predictions used the default precision `ap_fixed<16,6>` (16 bit signed with 6 integer bits). But was that the best choice? Below we inspect the distribution of values of the different parts of the model: data, thresholds, and scores.

Your exercise is to try some different choices of precision to see if we can better agreement between the `xgboost` and `conifer` predictions.
You can use the plot below to get some ideas for precision might work. Try changing both the width and integer parts, and maybe using rounding and saturation modes e.g. `ap_fixed<13,5,AP_RND_CONV,AP_SAT>` for 13 bits signed with 5 integer bits, convergent rounding (aka half-even rounding), and saturation.

Remake the performance comparison plots to see how things behave after your changes, and `build` the model again to see if there is any impact on resources & latency.

**Tip 1:** you can change the three precisions `InputPrecision`, `ThresholdPrecision`, and `ScorePrecision` indepdendently. Create the default configuration with those like this: `conifer.backends.xilinxhls.auto_config(granularity='full')`

**Tip 2:** you can create a new configuration, and apply it to the already converted model like this:
```python
new_config = conifer.backends.xilinxhls.auto_config(granularity='full')
new_config['OutputDir'] = 'some_sensible_directory_name'
... make some modifications
conifer_model = conifer.model.load_model('prj_conifer_part_1/my_prj.json', new_config=new_config)
```

In [ ]:
thresholds = np.array([t for trees_c in conifer_model.trees for tree in trees_c for t, f in zip(tree.threshold, tree.feature) if f != -2])
scores = np.array([v for trees_c in conifer_model.trees for tree in trees_c for v, f in zip(tree.value, tree.feature) if f == -2])

In [ ]:
po2_bins = 2**np.arange(-16,16,1,dtype='float')
bin_width = (po2_bins[1:] - po2_bins[:1])/3
hist_data, _ = np.histogram(np.abs(X_train.ravel()), bins=po2_bins, density=True)
hist_thresholds, _ = np.histogram(np.abs(thresholds), bins=po2_bins, density=True)
hist_scores, _ = np.histogram(np.abs(scores), bins=po2_bins, density=True)
ylim = np.max([hist_data.max(), hist_thresholds.max(), hist_scores.max()])

plt.figure()
plt.bar(po2_bins[:-1], hist_data, width=bin_width, alpha=0.5, label='data')
plt.bar(po2_bins[:-1], hist_thresholds, width=bin_width, alpha=0.5, label='thresholds')
plt.bar(po2_bins[:-1], hist_scores, width=bin_width, alpha=0.5, label='scores')
plt.plot([2**-10, 2**-10], [0,ylim*1.2], '--k', label='LSB')
plt.plot([2**5, 2**5], [0,ylim*1.2], '--b', label='MSB')
plt.ylim((1e-5, ylim*1.2))
plt.xscale('log', base=2)
plt.semilogy()
plt.legend()

In [ ]:
new_configuration = conifer.backends.xilinxhls.auto_config(granularity='full')
new_configuration['InputPrecision'] = ... # up to you